<a href="https://colab.research.google.com/github/LucasHenrique-dev/projeto-QTS/blob/main/desenvolvimento/GeradrorQTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Importar Bibliotecas

In [1]:
import time
import random
import math
import sys
import json
# libraries for the files in google drive
from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

## Importar Informações

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_id = '1Zv5i7SI4MMgMzSQjeBIS9q2AMAkbA0bX' #<-- You add in here the id from you google drive file, you can find it


download = drive.CreateFile({'id': file_id})


# Download the file to a local disc
download.GetContentFile('info_curso.json')

##Explorar Dados do Curso

In [3]:
arquivo = open('/content/info_curso.json')
curso = json.load(arquivo)

print(json.dumps(curso, indent=4))

{
    "horario_inicio": "7:00",
    "horario_fim": "12:00",
    "quantidade_turmas": 5,
    "disciplinas": {
        "SO": {
            "nome": "Sistemas Operacionais",
            "hora_aula": 1,
            "carga_horaria_semanal": 4
        },
        "MF": {
            "nome": "M\u00e9todos Formais",
            "hora_aula": 1,
            "carga_horaria_semanal": 4
        },
        "OC": {
            "nome": "Organiza\u00e7\u00e3o de Computadores",
            "hora_aula": 1,
            "carga_horaria_semanal": 4
        },
        "CO": {
            "nome": "Compiladores",
            "hora_aula": 1,
            "carga_horaria_semanal": 4
        },
        "IA": {
            "nome": "Intelig\u00eancia Artificial",
            "hora_aula": 1,
            "carga_horaria_semanal": 4
        }
    }
}


In [30]:
disciplinas = {}

disciplinas[0] = ('NA', 'Horário Vago')
for index, disciplina in enumerate(curso["disciplinas"].items()):
  disciplinas[index+1] = (disciplina[0], disciplina[1]["nome"])

disciplinas

{0: ('NA', 'Horário Vago'),
 1: ('SO', 'Sistemas Operacionais'),
 2: ('MF', 'Métodos Formais'),
 3: ('OC', 'Organização de Computadores'),
 4: ('CO', 'Compiladores'),
 5: ('IA', 'Inteligência Artificial')}

## Funções Auxiliares

In [4]:
def converter_minutos(hora):
  t = time.strptime(hora, '%H:%M')
  minutos = t[3] * 60 + t[4]
  return minutos

In [33]:
# NECESSÁRIO APROMORAR A FUNÇÃO PARA ACEITAR AULAS COMEÇANDO ÀS 5 DA MANHÃ E INDO ATÉ ÀS 4 DA MANHÃ
def calcular_disciplinas_dia(curso, segmentacao_horario):
  inicio, fim = curso['horario_inicio'], curso['horario_fim']
  inicio, fim = converter_minutos(inicio), converter_minutos(fim)
  return ((fim - inicio)/segmentacao_horario) + 1

In [6]:
def calcular_horas_vagas(disciplinas):
  return (''.join(disciplinas).replace("NA", ' ')).strip().count(" ")

In [7]:
dias_semana = ['Segunda','Terça','Quarta','Quinta','Sexta']

## Objetivo

O algoritmo visará montar um quadro de trabalho semanal contendo o menor **Tempo de Espera Entre Aulas** possível

Tempo de Espera Entre Aulas: horário de aula vaga entre aulas

##Definir Cromossomo

Definir a sequência genética de formação de um cromossomo

Os genes determinam aspectos de um indivíduo e o cromossomo é o conjunto dessas características o qual forma o indivíduo (possível solução do problema)

A ordem será os dias da semana contendo as disciplinas lecionadas no dia

[Segunda:(..disciplinas), Terça:(..disciplinas), Quarta:(..disciplinas), Quinta:(..disciplinas), Sexta:(..disciplinas)]

Ex.: [(SO,SO,SO,SO,MF,MF),(MF,MF,OC,OC,OC,OC),(CO,CO,CO,CO,IA,IA),(IA,IA,NA,NA,NA,NA),(NA,NA,NA,NA,NA,NA)]

In [34]:
segmentacao_horario = 60                                                          # Divisão do horário no QTS (minutos)
disciplinas_dia = int(calcular_disciplinas_dia(curso, segmentacao_horario))       # Retorna quantidade de sessões destinadas às disciplinas em um dia

##Impressão de Quadro de Trabalho Semanal

In [35]:
def imprimir_qts(calendario):
  dia_id = 0
  espera_total = 0
  for i in range((len(calendario) ) // disciplinas_dia):
      dia = dias_semana[i]
      comeco_semana = i*disciplinas_dia
      
      disciplinas_semana = calendario[comeco_semana:comeco_semana+disciplinas_dia]                        # coleta as disciplinas da semana
      ordem_disciplinas = [disciplinas[disciplina][0] for disciplina in disciplinas_semana]               # traduz números em disciplinas

      dia_id += 1                                                                                         # próximo dia da semana
      print(f'|{dia:7}| {", ".join(ordem_disciplinas)}|')
      
      espera_total += calcular_horas_vagas(ordem_disciplinas)

  print('\nTempo Total de Espera Entre Aulas: ', espera_total)

In [36]:
imprimir_qts([1,1,1,1,2,2, 2,2,3,3,3,3, 4,4,4,4,5,5, 5,5,0,0,0,0, 0,0,0,0,0,0])

|Segunda| SO, SO, SO, SO, MF, MF|
|Terça  | MF, MF, OC, OC, OC, OC|
|Quarta | CO, CO, CO, CO, IA, IA|
|Quinta | IA, IA, NA, NA, NA, NA|
|Sexta  | NA, NA, NA, NA, NA, NA|

Tempo Total de Espera Entre Aulas:  0


# Função de Avaliação (Fitness Function)
Definir algoritmo que atribua uma pontuação ao indivíduo gerado (possível solução) baseado no objetivo estabelecido

In [ ]:
def funcao_avaliacao(calendario):
  horas_vagas = 0
  primeira_aula = 0
  ultima_aula = 0

  for i in range(0, len(calendario), disciplinas_dia):
    aulas = calendario[i:i+disciplinas_dia]
    aulas_nao_vagas = [pos for pos, aula in enumerate(aulas, 0) if aula != 0]

    if len(aulas_nao_vagas) > 0:
      primeira_aula = aulas_nao_vagas[0]      #coleta o index da primeira aula não vaga
      ultima_aula = aulas_nao_vagas[-1]       #coleta o index da última aula não vaga
    else:
      primeira_aula = disciplinas_dia         #define a primeira aula como a última do dia
    
    # print("Inicio: ", primeira_aula, "\nFim: ", ultima_aula, "\nSemana: ", aulas[primeira_aula:ultima_aula+1])
    horas_vagas += aulas[primeira_aula:ultima_aula+1].count(0)
  
  return horas_vagas

In [ ]:
funcao_avaliacao([1,1,0,0,2,0, 2,2,0,3,0,3, 0,4,4,4,5,0, 5,5,0,0,0,0, 0,0,0,0,0,0])

4

##Criar o Domínio de Variação dos Genes

In [ ]:
dominio = [(0, len(disciplinas)-1)] * (len(dias_semana) * len(disciplinas))
dominio, len(dominio)

([(0, 5),
  (0, 5),
  (0, 5),
  (0, 5),
  (0, 5),
  (0, 5),
  (0, 5),
  (0, 5),
  (0, 5),
  (0, 5),
  (0, 5),
  (0, 5),
  (0, 5),
  (0, 5),
  (0, 5),
  (0, 5),
  (0, 5),
  (0, 5),
  (0, 5),
  (0, 5),
  (0, 5),
  (0, 5),
  (0, 5),
  (0, 5),
  (0, 5),
  (0, 5),
  (0, 5),
  (0, 5),
  (0, 5),
  (0, 5)],
 30)

# Mutação

In [ ]:
def mutacao(dominio, passo, calendario, probabilidade):
  gene = random.randint(0, len(dominio) - 1)      #Escolha do gene que sofrerá a mutação
  mutante = calendario
  #print("Gene: ", gene)
  #Joga um número aleatório para verificar se haverá mutação.
  if random.random() < probabilidade:
    if calendario[gene] != dominio[gene][0]:    #Verifica borda inferior
      mutante = calendario[0:gene] + [calendario[gene] - passo] + calendario[gene + 1:]
    elif calendario[gene] != dominio[gene][1]:  #Verifica borda superior
      mutante = calendario[0:gene] + [calendario[gene] + passo] + calendario[gene + 1:]
  return mutante

In [ ]:
mutacao(dominio, 2, [1,1,1,1,2,2, 2,2,3,3,3,3, 4,4,4,4,5,5, 5,5,0,0,0,0, 0,0,0,0,0,0], 0.9)

[1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2]

# Crossover

In [ ]:
def crossover(dominio, individuo1, individuo2):
  gene = random.randint(1, len(dominio) - 2)
  #print(gene)
  return individuo1[0:gene] + individuo2[gene:]

In [ ]:
s1 = [1,1,1,1,2,2, 2,2,3,3,3,3, 4,4,4,4,5,5, 5,5,0,0,0,0, 0,0,0,0,0,0]
s2 = [0,0,0,3,5,6, 1,1,1,1,5,5, 5,5,3,3,3,3, 1,1,2,2,2,2, 3,4,5,0,1,3]

In [ ]:
crossover(dominio, s1, s2)

[1,
 1,
 1,
 1,
 5,
 6,
 1,
 1,
 1,
 1,
 5,
 5,
 5,
 5,
 3,
 3,
 3,
 3,
 1,
 1,
 2,
 2,
 2,
 2,
 3,
 4,
 5,
 0,
 1,
 3]

# Algoritmo Genético Completo